# Building a csv of all your PBs
-- a short story by baldnate

# Get user id

In [ ]:
import json
import pandas as pd
import requests

users = {}
def getUserId(username):
  if username not in users:
    url = "https://www.speedrun.com/api/v1/users?name=" + username
    data = requests.get(url).json()['data']
    if len(data) == 1:
      users[username] = data[0]['id']
    else:
      raise Exception('Searched for ' + username + ', got back ' + str(len(data)) + ' entries (expected 1)') 
  return users[username]

userid = getUserId('baldnate')

# Get PBs

In [ ]:
def getPBs(userid):
  url = "https://www.speedrun.com/api/v1/users/" + userid + "/personal-bests?embed=game,category,region,platform,players"
  data = requests.get(url).json()['data']
  return data

pbs = getPBs(userid)
rawdf = pd.DataFrame(pbs)
runsdf = pd.DataFrame()

# Co-Op - aka: write player(s) to a column

# "Simple" Columns

In [ ]:

runsdf['place'] = rawdf['place']
runsdf['runid'] = rawdf.apply(lambda x: x.run['id'], axis=1)
runsdf['gameid'] = rawdf.apply(lambda x: x.run['game'], axis=1)
runsdf['gamename'] = rawdf.apply(lambda x: x.game['data']['names']['international'], axis=1)
runsdf['categoryname'] = rawdf.apply(lambda x: x.category['data']['name'], axis=1)
runsdf['time'] = rawdf.apply(lambda x: x.run['times']['primary_t'], axis=1)
runsdf['date'] = rawdf.apply(lambda x: x.run['date'], axis=1)
runsdf['video'] = rawdf.apply(lambda x: x.run['videos']['links'][0]['uri'], axis=1)
runsdf['comment'] = rawdf.apply(lambda x: str(x.run['comment']).replace('\n', ' ').replace('\r', ' ') , axis=1)

# Columns that need optional handling

In [ ]:
def getRegion(x):
  if x.region['data'] == []:
    return None
  else:
    return x.region['data']['name']

runsdf['regionname'] = rawdf.apply(lambda x: getRegion(x), axis=1)


In [ ]:
def getPlatform(x):
  if x.platform['data'] == []:
    return None
  else:
    return x.platform['data']['name']

runsdf['platformname'] = rawdf.apply(lambda x: getPlatform(x), axis=1)

# Sub-Categories

Memoized for speed and kindness.

In [ ]:
varMemo = {}

In [ ]:
def getVariable(variableid):
  if variableid not in varMemo:
    url = "https://www.speedrun.com/api/v1/variables/" + variableid
    response = requests.get(url)
    varMemo[variableid] = response.json()['data']
  return varMemo[variableid]

def getValue(variableid, valueid):
  var = getVariable(variableid)
  return var['values']['values'][valueid]['label']

def getSubCategories(x):
  if x.run['values'] == {}:
    return None
  else:
    vals = []
    for varid, valid in x.run['values'].items():
      if getVariable(varid)['is-subcategory']:
        vals.append(getValue(varid, valid))  
    return " -- ".join(vals)

runsdf['subcategories'] = rawdf.apply(lambda x: getSubCategories(x), axis=1)

# Dump to a csv

In [ ]:
def getPlayers(x):
  players = []
  for p in x.players['data']:
    players.append(p['names']['international']) 
  return ", ".join(players)

runsdf['players'] = rawdf.apply(lambda x: getPlayers(x), axis=1)

In [ ]:
import csv
runsdf.to_csv('runs.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)
print('csv exported')

In [ ]:
csvdf = pd.read_csv('runs.csv')
csvdf.head()